# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [2]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [7]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION =
    {
        'class': 'SimpleStrategy',
        'replication_factor': 1
    }
    """)
except  Exception as e:
    print(e)

#### Set Keyspace

In [8]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [9]:
# Delete and create table to match query 1
delete_session_query = "DROP TABLE IF EXISTS sessions"
session.execute(delete_session_query)
# Create session table with primary key = session_id and item_in_session
create_session_query = ("""
CREATE TABLE IF NOT EXISTS sessions(
artist text,
item_in_session int,
length float,
session_id int,
song_title text,
PRIMARY KEY (session_id,item_in_session)
)
""")
session.execute(create_session_query)
                    

In [10]:
#Insert all sessions from the csv data into the sessions table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO sessions (artist, item_in_session, length, session_id, song_title)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (line[0], int(line[3]), float(line[5]), int(line[8]), line[9]))

#### Do a SELECT to verify that the data have been inserted into each table

In [11]:
#Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4
select_query =  "select artist, song_title, length from sessions WHERE session_id = 338 and item_in_session = 4"
rows = session.execute(select_query)
for row in rows:
    print(row.length, row.artist, row.song_title)

495.30731201171875 Faithless Music Matters (Mark Knight Dub)


In [12]:
#Delete and create the users table to match query 2
delete_users_query = "DROP TABLE IF EXISTS users"
session.execute(delete_users_query)
#Create users table with primary key = user_id and session_id
create_users_query = ("""
CREATE TABLE IF NOT EXISTS users(
artist text,
first_name text,
item_in_session int,
last_name text,
session_id int,
song_title text,
user_id int,
PRIMARY KEY ((user_id, session_id), item_in_session)
)
""")
session.execute(create_users_query)               

In [13]:
#Insert all users into the users table
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = "INSERT INTO users (artist, first_name, item_in_session, last_name, session_id, song_title, user_id)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[1], int(line[3]), line[4], int(line[8]), line[9], int(line[10]))) 

In [14]:
#Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
select_user_query = "select artist, song_title, first_name, last_name from users WHERE session_id = 182 and user_id = 10"
rows = session.execute(select_user_query)
for row in rows:
    print (row.artist, row.song_title, row.first_name, row.last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [15]:
#Delete and create the song_listens table to match query 3.
delete_song_listens_query = "DROP TABLE IF EXISTS song_listens"
session.execute(delete_song_listens_query)
#Create song_listens table with primary key = song_title and user_id
create_song_listens_query = ("""
CREATE TABLE IF NOT EXISTS song_listens(
last_name text,
first_name text,
song_title text,
user_id int,
PRIMARY KEY (song_title, user_id)
)
""")
session.execute(create_song_listens_query)

In [16]:
#Insert all song_listens into the table
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO song_listens (first_name, last_name, song_title, user_id)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[1], line[4], line[9], int(line[10]))) 

In [17]:
#Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
select_song_query =  "select first_name, last_name from song_listens WHERE song_title = 'All Hands Against His Own'"
rows = session.execute(select_song_query)
for row in rows:
    print(row.first_name, row.last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [18]:
delete_sessions_query = "DROP TABLE IF EXISTS sessions"
session.execute(delete_sessions_query)

delete_users_query = "DROP TABLE IF EXISTS users"
session.execute(delete_users_query)

delete_song_listens_table_query = "DROP TABLE IF EXISTS song_listens"
session.execute(delete_song_listens_query)

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()